In [ ]:
%cd /content/drive/MyDrive/src/

In [ ]:
#!tar zxvf cifar100.tar.gz

In [125]:
!python test.py

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:124: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
processing:0/79
acc:0.7212, evaluate time:220.634626 sec


In [ ]:
!python static_quantize.py

In [ ]:
!python QAT.py

In [ ]:
import torch

state_dict1 = torch.load("../pth/resnet56_tuned20.pth")
state_dict2 = torch.load("../pth/resnet56_quantized20.pth")
state_dict3 = torch.load("./test.pth")

"""
print(type(state_dict1))
i = 1
for k in state_dict1:
  print(f"key {i}: {k}")
  print(type(state_dict1[k]), state_dict1[k].size())
  print(state_dict1[k][0])
  i += 1
  if i >= 2:
    break
"""

print(type(state_dict2))
i = 1
for k in state_dict2:
  print(f"key {i}: {k}")
  print(type(state_dict2[k]))
  print(state_dict2[k])
  i += 1
  if i >= 2:
    break

print(type(state_dict3))
i = 1
for k in state_dict3:
  print(f"key {i}: {k}")
  print(type(state_dict3[k]))
  print(state_dict3[k])
  i += 1
  if i >= 2:
    break

In [ ]:
import torch
import torch.nn as nn

def print_key(model, num=5):
  _ = 0
  keys = []
  for k in model.state_dict():
    keys.append(f"key{_+1}: " + k)
    _ += 1
    if _ >= num:
      break
  print(keys)

class test(nn.Module):
  def __init__(self):
    super(test, self).__init__()
    self.lin1 = nn.Linear(10, 5)
    self.act1 = nn.ReLU()
    self.lin2 = nn.Linear(5, 3)

    self.quant = torch.quantization.QuantStub()
    self.dequant = torch.quantization.DeQuantStub()

  def forward(self, x):
    x = self.quant(x)
    x = self.lin1(x)
    x = self.act1(x)
    x = self.lin2(x)
    x = self.dequant(x)
    return x

m = test()
m.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(m, inplace=True)
torch.quantization.convert(m.eval(), inplace=True)
torch.save(m.state_dict(), "./test1.pth")
torch.save(m, "./test1.mod")

print("m")
print(m)
print(m.state_dict())

m2 = torch.load("./test1.mod")
print("m2")
print(m2)
print(m2.state_dict())
